In [1]:
import sqlite3 as sl
import pandas as pd
from datetime import datetime


def connect_to_sqlite3(filepath: str) -> sl.Connection:
    return sl.connect(filepath)


def insert(table: str, con: sl.Connection, df: pd.DataFrame, if_exists: str = 'append', chunksize: int = 1000) -> None:
    # This will need exception handling but not sure how yet.
    df.to_sql(table, con, if_exists=if_exists, chunksize=chunksize)


def fetch(con: sl.Connection, query: str) -> pd.DataFrame:
    return pd.read_sql(query, con)


conn = connect_to_sqlite3('../db/ghs.db')

In [2]:
data = fetch(conn, '''
WITH a AS (
  SELECT
    channel_handle,
    json_extract(raw_post, '$.fwd_from') AS fwd_from
  FROM
    raw_posts rp
  WHERE
    fwd_from IS NOT NULL
)
SELECT
  channel_handle forwarder,
  json_extract(fwd_from, '$.from_entity_url') src
FROM
  a
''')
print(data.shape)
display(data.head(1))

(404484, 2)


,forwarder,src
0,justicseeker,https://t.me/officalnews121


In [3]:
data = data.loc[~data.src.isna()]
print(data.shape)
data['src'] = [l.split('/')[-1] for l in data.src]
data.columns = ['target', 'source']
data = data[['source', 'target']]
data.head()

(381318, 2)


,source,target
0,officalnews121,justicseeker
1,mindoyesayargyi000,justicseeker
2,mindoyesayargyi000,justicseeker
3,banyunt969,justicseeker
4,banyunt969,justicseeker


# Insert into NEO4J

In [4]:
'''
CREATE (c:City {name: "Zagreb", population_size: 1000000});
'''


def generate_cypher_create_node(d: pd.DataFrame):
    q = 'CREATE ' + ','.join((
        f'(:{r.node_type}'
        ' {'
        f'name:"{r.channel_handle}"'
        '})'
    ) for _,r in d.iterrows())

    now = datetime.now().strftime('%Y%m%d%H%M')
    with open(f'../neo4j/nodes_{now}.cypher', 'w') as f:
        f.write(q)

    return q

In [5]:
names = tuple(set(data.values.flatten()))
node_types = ['TG_CHANNEL'] * len(names)

nodes = pd.DataFrame({
    'channel_handle': names,
    'node_type': node_types
})
print(len(nodes))
display(nodes.head())

851


,channel_handle,node_type
0,stars_over_burma_education,TG_CHANNEL
1,springlotto,TG_CHANNEL
2,ashindelicacy,TG_CHANNEL
3,kbk303,TG_CHANNEL
4,mmsub18plu,TG_CHANNEL


In [6]:
_ = generate_cypher_create_node(nodes)

## Generate Relationships

In [7]:
# 'MATCH (t:table),(m:model) WHERE t.tid = "T0526" AND m.mid = "M001" CREATE (t)-[R00865:APPEARS_IN]->(m);'

def create_rs(d: pd.DataFrame):
    q = ';'.join(
        (
        f'MATCH (s:TG_CHANNEL),(t:TG_CHANNEL) WHERE s.name="{r.source}" AND t.name="{r.target}" CREATE (s)-[:FORWARDED_BY'
        '{weight:' + str(r.weight) + ''
        '}]->(t)'
        )
    for _,r in d.iterrows())
    
    now = datetime.now().strftime('%Y%m%d%H%M')
    with open(f'../neo4j/rs_{now}.cypher', 'w') as f:
        f.write(q)

    return q

In [8]:
data.head()

,source,target
0,officalnews121,justicseeker
1,mindoyesayargyi000,justicseeker
2,mindoyesayargyi000,justicseeker
3,banyunt969,justicseeker
4,banyunt969,justicseeker


In [9]:
forwards = data.groupby(['source','target']).size().reset_index()
forwards.columns = ['source','target','weight']
forwards

,source,target,weight
0,-,forwardchannelrangoon,6
1,-,kyawswar9690,5
2,-,mgtn_spring_and_scout_news,4
3,-,mmsoldier,1
4,-,msi22222,3
...,...,...,...
4040,zscoutfighter,no_name_myanmar,496
4041,zscoutfighter,pcdmchannel,3
4042,zscoutfighter,sggscoutnetwork,6
4043,zscoutfighter,tktdpscoutnews,6


In [10]:
_ = create_rs(forwards)